In [ ]:
!pip install selenium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00


In [ ]:
!pip install webdriver-manager

In [ ]:
from selenium import webdriver
import time
from selenium.webdriver import Chrome
from webdriver_manager.chrome import ChromeDriverManager


In [ ]:
!apt-get install chromium-driver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'chromium-chromedriver' instead of 'chromium-driver'
The following additional packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 libudev1 snapd squashfs-tools systemd-hwe-hwdb
  udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser chromium-chromedriver libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
The following packages will be upgraded:
  libudev1
1 upgraded, 9 newly installed, 0 to remove and 48 not upgraded.
Need to get 28.5 MB of archives.
After this operation, 118 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.3 [595 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://ar

In [ ]:
def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--verbose")
    options.add_argument('--no-sandbox')
    options.add_argument('--headless')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920, 1200")
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

In [ ]:
driver=web_driver()

In [ ]:
number  = [
    3034, 2454, 2357, 2395, 3443, 3231, 9921, 2382, 2609, 2406,
    2327, 2376, 3450, 2377, 2365, 2474, 3037, 2204, 2356, 3227,
    3105, 3045, 2308, 3702, 3661, 2330, 3044, 2347, 6305, 3289,
    3032, 1597, 5871, 3653, 8069, 1102, 3008, 3023, 6415, 2448,
    2882, 2615, 2543, 6239, 3711, 6223, 8046, 1402, 2881, 4938,
    4575, 2884, 3432, 3474, 4904, 6146, 2887, 6641, 2317, 3035,
    2458, 2301, 2492, 2013, 4958, 1605, 3665, 6422, 3006, 2367,
    1503, 2449, 2606, 6139, 2809, 6269, 9933, 2912, 2408, 1722,
    1216, 2428, 2451, 2353, 9914, 2393, 4961, 9955, 2885, 2886,
    2388
]
year = [113, 112, 111, 110, 109, 108, 107]

In [ ]:
number  = [
    6139, 2809, 6269, 9933, 2912, 2408, 1722,
    1216, 2428, 2451, 2353, 9914, 2393, 4961, 9955, 2885, 2886,
    2388
]
year = [113, 112, 111, 110, 109, 108, 107]

In [ ]:
import requests
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import pandas as pd
df_NOPDF = pd.DataFrame(columns=['公司代號', '年'])
df_ERROR = pd.DataFrame(columns=['公司代號', '年'])

driver=web_driver()
for company_code in number:
    for y in year:
        url = f"https://doc.twse.com.tw/server-java/t57sb01?step=1&colorchg=1&co_id={company_code}&year={y}&mtype=F&"
        driver.get(url)

        try:
            # Use WebDriverWait to handle dynamic content
            containers = driver.find_elements(By.XPATH, '//a[contains(., "F04") and contains(@href, ".pdf")]')
            if not containers:
                print(f"No PDF link found for company {company_code} in year {y}")
                df_NOPDF = pd.concat([df_NOPDF, pd.DataFrame([{'公司代號': company_code, '年': y}])], ignore_index=True)
                continue

            containers = driver.find_elements(By.XPATH,'//a[contains(.,"F04") and contains(@href,".pdf")]')[0].get_attribute('href')
            pdf_name = containers.split('\"')[-2]
            driver.find_element(By.XPATH,f"//a[contains(@href,'{containers}')]").click()

            windows=driver.window_handles  #獲得當前瀏覽器所有視窗
            driver.switch_to.window(windows[-1]) #換到最新的視窗
            time.sleep(3) #休息3s
            url = driver.current_url #看看目前的url
            time.sleep(3)
            right_click = driver.find_element(By.LINK_TEXT,pdf_name)
            ActionChains(driver).double_click(right_click).perform() #左擊
            windows=driver.window_handles  #獲得當前瀏覽器所有視窗
            driver.switch_to.window(windows[-1]) #換到最新的視窗
            time.sleep(3) #休息3s
            url = driver.current_url #看看目前的url

            # Download the PDF
            file_path = f"/content/drive/MyDrive/Colab Notebooks/年報/{company_code}_{y}.pdf"
            response = requests.get(url)

            if response.status_code == 200:
                with open(file_path, 'wb') as file:
                    file.write(response.content)
                print(f"PDF downloaded: {file_path}")
            else:
                print(f"Failed to download PDF for company {company_code} in year {y}")

            # Close the current window and switch back to the original one
            driver.close()
            driver.switch_to.window(windows[0])

        except Exception as e:
            print(f"An error occurred: {e}")
            df_ERROR = pd.concat([df_ERROR, pd.DataFrame([{'公司代號': company_code, '年': y}])], ignore_index=True)
            continue

driver.quit()
print('Crawling completed')
print(df_NOPDF)
print(df_ERROR)

PDF downloaded: /content/drive/MyDrive/Colab Notebooks/年報/6139_113.pdf
PDF downloaded: /content/drive/MyDrive/Colab Notebooks/年報/6139_112.pdf
PDF downloaded: /content/drive/MyDrive/Colab Notebooks/年報/6139_111.pdf
PDF downloaded: /content/drive/MyDrive/Colab Notebooks/年報/6139_110.pdf
PDF downloaded: /content/drive/MyDrive/Colab Notebooks/年報/6139_109.pdf
PDF downloaded: /content/drive/MyDrive/Colab Notebooks/年報/6139_108.pdf
PDF downloaded: /content/drive/MyDrive/Colab Notebooks/年報/6139_107.pdf
PDF downloaded: /content/drive/MyDrive/Colab Notebooks/年報/2809_113.pdf
PDF downloaded: /content/drive/MyDrive/Colab Notebooks/年報/2809_112.pdf
PDF downloaded: /content/drive/MyDrive/Colab Notebooks/年報/2809_111.pdf
PDF downloaded: /content/drive/MyDrive/Colab Notebooks/年報/2809_110.pdf
PDF downloaded: /content/drive/MyDrive/Colab Notebooks/年報/2809_109.pdf
PDF downloaded: /content/drive/MyDrive/Colab Notebooks/年報/2809_108.pdf
PDF downloaded: /content/drive/MyDrive/Colab Notebooks/年報/2809_107.pdf
PDF do

In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 38.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import pdfplumber #需安裝 : https://pypi.org/project/pdfplumber/
list_company=pd.DataFrame()
file_path = f"/content/drive/MyDrive/Colab Notebooks/年報"
for i in number:
  for j in year:
    try:
      pdf_name = f'{i}_{j}.pdf'
      pdf = pdfplumber.open(pdf_name)
      for p in range(1,5):
        p = pdf.pages[p]
        table=p.extract_text()       #讀文字
        print(table)
        print('*****************************************')
    except Exception as e:
      continue